In [2]:
import os
import re
import aocd
import string
import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from copy import deepcopy
from aocd import submit


#Part 1
data = aocd.get_data(day=12, year=2023).split('\n')

In [14]:
data = """???.### 1,1,3
.??..??...?##. 1,1,3
?#?#?#?#?#?#?#? 1,3,1,6
????.#...#... 4,1,1
????.######..#####. 1,6,5
?###???????? 3,2,1""".split('\n')

In [15]:



def is_hash_at(i, placement, contig_damaged, contig_damaged_cumsum):

    end = np.array(placement) + contig_damaged_cumsum
    start = end - contig_damaged
    #print(start,end)
    for j in range(len(end)):
        if end[j] > i:
            #print(i,end[j],start[j])
            if start[j] <= i:
                return True
            else:
                return False


In [16]:
# part 1

_sum = 0
for line in tqdm(data):
    print(line)
    condition, contig_damaged = line.split(' ')
    #print('\nLine:',condition, contig_damaged)
    contig_damaged = contig_damaged.split(',')
    contig_damaged = [int(x) for x in contig_damaged]
        
    # replace dots next to eachother with single dot.
    condition = re.sub('\.+', '.', condition)
    condition = re.sub('^\.+','',condition)
    condition = re.sub('\.+$','',condition)
    
    total = len(condition)
    no_damaged = sum(contig_damaged)
    no_undamaged = total - no_damaged
    
    no_known_damaged = len([x for x in condition if x == '#'])
    
    no_unknown_damaged = no_damaged - no_known_damaged
    no_unknown = len([x for x in condition if x == '?'])
    no_unknown_undamaged = no_unknown - no_unknown_damaged
    
    possible_placements = list(range(no_undamaged + 1))

    possible_placements = list(itertools.combinations(possible_placements,len(contig_damaged)))
    print(len(possible_placements))
    contig_damaged_cumsum = np.cumsum(contig_damaged)
    for i in range(len(condition)):
        char = condition[i]
        #print(char)
        if char == '?':
            pass
        elif char == '.':
            for placement in list(possible_placements):
                if is_hash_at(i, placement, contig_damaged, contig_damaged_cumsum):
                    possible_placements.remove(placement)
        elif char == '#':
            for placement in list(possible_placements):
                #print(i, placement)
                if not is_hash_at(i, placement, contig_damaged, contig_damaged_cumsum):
                    #print('deleting...', placement)
                    possible_placements.remove(placement)
    
    print(len(possible_placements))
    _sum += len(possible_placements)
    break
_sum

  0%|          | 0/6 [00:00<?, ?it/s]

???.### 1,1,3
1
1


1

In [62]:
# Part 2

# based on https://github.com/clrfl/AdventOfCode2023/blob/master/12/explanation.ipynb

data = aocd.get_data(day=12, year=2023).split('\n')


def move_forward(values, counts, possible_transitions):
    new_counts = {key: 0 for key in counts}
    for key in counts:
        transitions = possible_transitions[key]
        for t in transitions:
            if t[0] in values:
                new_counts[t] += counts[key]
    return new_counts

def transition_to_dot(counts, possible_transitions):
    return move_forward(['.'], counts, possible_transitions)

def transition_to_hash(counts, possible_transitions):
    return move_forward(['#'], counts, possible_transitions)

def transition_to_dot_or_hash(counts, possible_transitions):
    return move_forward(['.','#'], counts, possible_transitions)

_sum = 0
for line in tqdm(data):
    condition, contig_damaged = line.split(' ')
    #print('\nLine:',condition, contig_damaged)
    contig_damaged = contig_damaged.split(',')
    contig_damaged = [int(x) for x in contig_damaged]

    
    condition = ((condition + '?')*5)[:-1]
    contig_damaged = contig_damaged*5
    
    possible_states = []
    possible_transitions = {}
    counts = {}
    damaged_part_no = 0
    no_damaged_groups = len(contig_damaged)
    for group_i in range(no_damaged_groups):
        possible_states += [f'._{group_i}']
        possible_transitions[f'._{group_i}'] = [f'._{group_i}', f'#_{damaged_part_no}']
        counts[f'._{group_i}'] = 0
        
        no_damaged_in_group_i = contig_damaged[group_i]
        for k in range(no_damaged_in_group_i):
            #print(no_damaged_in_group_i, k)
            possible_states += [f'#_{damaged_part_no}']
            
            if k == no_damaged_in_group_i - 1:
                possible_transitions[f'#_{damaged_part_no}'] = [f'._{group_i + 1}']
            else:
                possible_transitions[f'#_{damaged_part_no}'] = [f'#_{damaged_part_no + 1}']
            
            counts[f'#_{damaged_part_no}'] = 0
            
            damaged_part_no += 1
    end_states = [f'._{group_i+1}', f'#_{damaged_part_no - 1}']
    possible_states += [f'._{group_i+1}']
    possible_transitions[f'._{group_i + 1}'] = [f'._{group_i +1 }']
    counts[f'._{group_i + 1}'] = 0
            

    
    
    #print('\nLine',condition, contig_damaged)
    #print(' Possible States: ', possible_states)
    #print(' Possible Transitions: ', possible_transitions)
    #print(' End States: ', end_states)
    
    counts['._0'] = 1
    l = 0
    for char in condition:
        if l == 10:
            pass#break
        l += 1
        #print('\nBefore: ',char, {k:v for k,v in counts.items() if v > 0})
        if char == '?':
            counts = transition_to_dot_or_hash(counts, possible_transitions)
        elif char == '.':
            counts = transition_to_dot(counts, possible_transitions)
        else:
            counts = transition_to_hash(counts, possible_transitions)
        #print('After:    ',{k:v for k,v in counts.items() if v > 0})
    no_possible_ends = sum([counts[end_state] for end_state in end_states])
    _sum += no_possible_ends
    print('Number possible ends', no_possible_ends)
_sum

 12%|█▏        | 118/1000 [00:00<00:01, 554.45it/s]

Number possible ends 14763
Number possible ends 16
Number possible ends 5772
Number possible ends 1024
Number possible ends 75769903
Number possible ends 1
Number possible ends 14776336
Number possible ends 4032
Number possible ends 81
Number possible ends 24576
Number possible ends 32
Number possible ends 12907776
Number possible ends 16
Number possible ends 16
Number possible ends 1
Number possible ends 32
Number possible ends 512
Number possible ends 768
Number possible ends 9284399
Number possible ends 5361664
Number possible ends 1875
Number possible ends 2500
Number possible ends 60000
Number possible ends 17082
Number possible ends 30045015
Number possible ends 125358
Number possible ends 16384
Number possible ends 16878
Number possible ends 7776
Number possible ends 2476099
Number possible ends 162
Number possible ends 3125
Number possible ends 60444
Number possible ends 704349
Number possible ends 243
Number possible ends 100000
Number possible ends 71150000
Number possible en

 25%|██▌       | 251/1000 [00:00<00:01, 604.18it/s]


Number possible ends 3981312
Number possible ends 162
Number possible ends 1875
Number possible ends 1027369
Number possible ends 1
Number possible ends 768
Number possible ends 345744
Number possible ends 701220
Number possible ends 154116
Number possible ends 1
Number possible ends 9402
Number possible ends 1875
Number possible ends 512
Number possible ends 46972500
Number possible ends 3699522
Number possible ends 15384704
Number possible ends 607500
Number possible ends 11033792288
Number possible ends 768
Number possible ends 14116
Number possible ends 16
Number possible ends 4221
Number possible ends 1
Number possible ends 768
Number possible ends 243
Number possible ends 32768
Number possible ends 5184
Number possible ends 29755
Number possible ends 596958
Number possible ends 7776
Number possible ends 1875
Number possible ends 3888
Number possible ends 15653300
Number possible ends 2240000
Number possible ends 2970240
Number possible ends 32
Number possible ends 32
Number poss

 36%|███▌      | 362/1000 [00:00<00:01, 579.03it/s]

...

 97%|█████████▋| 969/1000 [00:01<00:00, 608.96it/s]

Number possible ends 32
Number possible ends 384160
Number possible ends 266333276
Number possible ends 16
Number possible ends 53283198
Number possible ends 32
Number possible ends 243
Number possible ends 32
Number possible ends 516795935
Number possible ends 2712
Number possible ends 15892
Number possible ends 471108
Number possible ends 3641712
Number possible ends 32
Number possible ends 213945504
Number possible ends 243
Number possible ends 243
Number possible ends 1
Number possible ends 5004909
Number possible ends 5468750
Number possible ends 4485652
Number possible ends 327680
Number possible ends 768
Number possible ends 7572
Number possible ends 81
Number possible ends 162
Number possible ends 81
Number possible ends 186624
Number possible ends 32
Number possible ends 1024
Number possible ends 53124
Number possible ends 246535
Number possible ends 22228
Number possible ends 6599371
Number possible ends 27108
Number possible ends 2500
Number possible ends 1024
Number possibl

100%|██████████| 1000/1000 [00:01<00:00, 592.46it/s]

Number possible ends 4567752
Number possible ends 12865920
Number possible ends 585712016972
Number possible ends 105415
Number possible ends 1250
Number possible ends 2500
Number possible ends 712070
Number possible ends 162
Number possible ends 27799
Number possible ends 59049
Number possible ends 1950176
Number possible ends 10952456
Number possible ends 768
Number possible ends 16
Number possible ends 32
Number possible ends 3125
Number possible ends 24576
Number possible ends 4077631408
Number possible ends 24605
Number possible ends 1
Number possible ends 9604
Number possible ends 83893911
Number possible ends 512
Number possible ends 187800
Number possible ends 128205624
Number possible ends 512
Number possible ends 3987585
Number possible ends 59049
Number possible ends 32
Number possible ends 124416
Number possible ends 768


1738259948652